In [ ]:
# Example - https://github.com/cornelliusyudhawijaya/Churn_Causality_Analysis/blob/main/Causal%20Analysis%20Do%20Why.ipynb
    
import numpy as np
import pandas as pd

from dowhy import CausalModel
import dowhy.datasets 

# Avoid printing dataconversion warnings from sklearn and numpy
import warnings
from sklearn.exceptions import DataConversionWarning
warnings.filterwarnings('ignore')

In [ ]:
from sklearn import preprocessing
dataset_orig = pd.read_csv('../../../data/MEPS/h192.csv')

# ## Drop NULL values
dataset_orig = dataset_orig.dropna()


dataset_orig = dataset_orig.rename(columns = {'FTSTU53X' : 'FTSTU', 'ACTDTY53' : 'ACTDTY', 'HONRDC53' : 'HONRDC', 'RTHLTH53' : 'RTHLTH',
                              'MNHLTH53' : 'MNHLTH', 'CHBRON53' : 'CHBRON', 'JTPAIN53' : 'JTPAIN', 'PREGNT53' : 'PREGNT',
                              'WLKLIM53' : 'WLKLIM', 'ACTLIM53' : 'ACTLIM', 'SOCLIM53' : 'SOCLIM', 'COGLIM53' : 'COGLIM',
                              'EMPST53' : 'EMPST', 'REGION53' : 'REGION', 'MARRY53X' : 'MARRY', 'AGE53X' : 'AGE',
                              'POVCAT16' : 'POVCAT', 'INSCOV16' : 'INSCOV'})


dataset_orig = dataset_orig[dataset_orig['PANEL'] == 21]
dataset_orig = dataset_orig[dataset_orig['REGION'] >= 0] # remove values -1
dataset_orig = dataset_orig[dataset_orig['AGE'] >= 0] # remove values -1
dataset_orig = dataset_orig[dataset_orig['MARRY'] >= 0] # remove values -1, -7, -8, -9
dataset_orig = dataset_orig[dataset_orig['ASTHDX'] >= 0] # remove values -1, -7, -8, -9
dataset_orig = dataset_orig[(dataset_orig[['FTSTU','ACTDTY','HONRDC','RTHLTH','MNHLTH','HIBPDX','CHDDX','ANGIDX','EDUCYR','HIDEG',
                         'MIDX','OHRTDX','STRKDX','EMPHDX','CHBRON','CHOLDX','CANCERDX','DIABDX',
                         'JTPAIN','ARTHDX','ARTHTYPE','ASTHDX','ADHDADDX','PREGNT','WLKLIM',
                         'ACTLIM','SOCLIM','COGLIM','DFHEAR42','DFSEE42','ADSMOK42',
                         'PHQ242','EMPST','POVCAT','INSCOV']] >= -1).all(1)]

# ## Change symbolics to numerics
dataset_orig['RACEV2X'] = np.where((dataset_orig['HISPANX'] == 2 ) & (dataset_orig['RACEV2X'] == 1), 1, dataset_orig['RACEV2X'])
dataset_orig['RACEV2X'] = np.where(dataset_orig['RACEV2X'] != 1 , 0, dataset_orig['RACEV2X'])
dataset_orig = dataset_orig.rename(columns={"RACEV2X" : "RACE"})
# dataset_orig['UTILIZATION'] = np.where(dataset_orig['UTILIZATION'] >= 10, 1, 0)



def utilization(row):
        return row['OBTOTV16'] + row['OPTOTV16'] + row['ERTOT16'] + row['IPNGTD16'] + row['HHTOTD16']

dataset_orig['TOTEXP16'] = dataset_orig.apply(lambda row: utilization(row), axis=1)
lessE = dataset_orig['TOTEXP16'] < 10.0
dataset_orig.loc[lessE,'TOTEXP16'] = 0.0
moreE = dataset_orig['TOTEXP16'] >= 10.0
dataset_orig.loc[moreE,'TOTEXP16'] = 1.0

dataset_orig = dataset_orig.rename(columns = {'TOTEXP16' : 'UTILIZATION'})

dataset_orig = dataset_orig[['REGION','AGE','SEX','RACE','MARRY',
                                 'FTSTU','ACTDTY','HONRDC','RTHLTH','MNHLTH','HIBPDX','CHDDX','ANGIDX',
                                 'MIDX','OHRTDX','STRKDX','EMPHDX','CHBRON','CHOLDX','CANCERDX','DIABDX',
                                 'JTPAIN','ARTHDX','ARTHTYPE','ASTHDX','ADHDADDX','PREGNT','WLKLIM',
                                 'ACTLIM','SOCLIM','COGLIM','DFHEAR42','DFSEE42', 'ADSMOK42',
                                 'PCS42','MCS42','K6SUM42','PHQ242','EMPST','POVCAT','INSCOV','UTILIZATION', 'PERWT16F']]

dataset_orig = dataset_orig.rename(columns={"UTILIZATION": "Probability","RACE" : "race"})
protected_attribute = 'race'


from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
dataset_orig = pd.DataFrame(scaler.fit_transform(dataset_orig),columns = dataset_orig.columns)

dataset_orig_male , dataset_orig_female = [x for _, x in dataset_orig.groupby(dataset_orig[protected_attribute] == 0)]

print(dataset_orig.head(5))

# Fairway

In [ ]:
from sklearn.linear_model import LogisticRegression

dataset_orig_male[protected_attribute] = 0
X_train_male, y_train_male = dataset_orig_male.loc[:, dataset_orig_male.columns != 'Probability'], dataset_orig_male['Probability']
clf_male = LogisticRegression(C=1.0, penalty='l2', solver='liblinear', max_iter=100)
clf_male.fit(X_train_male, y_train_male)

In [ ]:
X_train_female, y_train_female = dataset_orig_female.loc[:, dataset_orig_female.columns != 'Probability'], dataset_orig_female['Probability']
clf_female = LogisticRegression(C=1.0, penalty='l2', solver='liblinear', max_iter=100)
clf_female.fit(X_train_female, y_train_female)

In [ ]:
print(dataset_orig.shape)

df_removed = pd.DataFrame(columns=dataset_orig.columns)

for index,row in dataset_orig.iterrows():
    row_ = [row.values[0:len(row.values)-1]]
    y_male = clf_male.predict(row_)
    y_female = clf_female.predict(row_)
    if y_male[0] != y_female[0]:        
        df_removed = df_removed.append(row, ignore_index=True)
        dataset_orig = dataset_orig.drop(index)
        
print(dataset_orig.shape)
print(df_removed.shape)

In [ ]:
dataset_orig

In [ ]:
causal_graph = """
digraph {
SEX;
AGE;
race;
MARRY;
REGION;
Probability;
U[label="Unobserved Confounders"];
SEX -> Probability;
race -> REGION; REGION -> Probability;
race -> Probability;
AGE -> Probability;
SEX -> MARRY; MARRY -> Probability;
U->SEX;U->race;U->Probability;
}
"""

## dowhy works for binary inputs only


dataset_orig['Probability'] = np.where(dataset_orig['Probability'] == 0, False, True)
dataset_orig[protected_attribute] = np.where(dataset_orig[protected_attribute] == 1, True, False)

In [ ]:
# With graph
model=CausalModel(
        data = dataset_orig,
        treatment=protected_attribute,
        outcome="Probability",
        graph=causal_graph.replace("\n", " ")
        )
model.view_model()

In [ ]:
identified_estimand = model.identify_effect(proceed_when_unidentifiable=True)
print(identified_estimand)

In [ ]:
causal_estimate = model.estimate_effect(identified_estimand,
        method_name="backdoor.propensity_score_stratification")
print(causal_estimate)
print("Causal Estimate is " + str(causal_estimate.value))

In [ ]:
# Causal effect on the control group (ATC)
causal_estimate_att = model.estimate_effect(identified_estimand,
        method_name="backdoor.propensity_score_stratification",
        target_units = "atc")
print(causal_estimate_att)
print("Causal Estimate is " + str(causal_estimate_att.value))

# Add Random Common Cause

In [ ]:
refutel = model.refute_estimate(identified_estimand,causal_estimate, "random_common_cause")
print(refutel)

# Replace Treatment with Placebo

In [ ]:
refutel = model.refute_estimate(identified_estimand,causal_estimate,method_name="placebo_treatment_refuter",placebo_type="permute")
print(refutel)

# Remove Random Subset of Data

In [ ]:
refutel = model.refute_estimate(identified_estimand,causal_estimate, "data_subset_refuter")
print(refutel)